# Example Tools

In [ ]:
#| default_exp tools

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
from stringdale.core import get_git_root, load_env, checkLogs
import pytest

In [ ]:
#| hide
load_env()

True

In [ ]:
# | export
import os
from pathlib import Path
from copy import deepcopy
from fastcore.foundation import L
import json

from collections import defaultdict
from singleton_decorator import singleton

import logging

logger = logging.getLogger(__name__)

from openai import OpenAI
import base64
from typing import Optional

from joblib import Memory
from pathlib import Path

from stringdale.core import disk_cache



## Wikipedia search

In [ ]:
#| export
import wikipedia

In [ ]:
# | export

@disk_cache.cache
def wikipedia_search(q:str):
    """
    A tool to query wikipedia, useful when you need to find information about a specific topic or person that is well known.
    Useful when you dont have enough context to reason about how to answer the question.

    Args:
        q (str): The query string to search for

    Returns:
        str: The wikipedia search results
    """

    return wikipedia.page(q).content

In [ ]:
input = {'q':'barak obama'}

In [ ]:
wikipedia_search(**input)

'Barack Hussein Obama II (born August 4, 1961) is an American politician who was the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, he was the first African American president. Obama previously served as a U.S. senator representing Illinois from 2005 to 2008 and as an Illinois state senator from 1997 to 2004.\nBorn in Honolulu, Hawaii, Obama graduated from Columbia University in 1983 with a Bachelor of Arts degree in political science and later worked as a community organizer in Chicago. In 1988, Obama enrolled in Harvard Law School, where he was the first black president of the Harvard Law Review. He became a civil rights attorney and an academic, teaching constitutional law at the University of Chicago Law School from 1992 to 2004. In 1996, Obama was elected to represent the 13th district in the Illinois Senate, a position he held until 2004, when he successfully ran for the U.S. Senate. In the 2008 presidential election, after a close primar

## Google search

In [ ]:
# | export

from serpapi import SerpApiClient


In [ ]:
def get_serper_api_key():
    key = os.getenv('SERPER_API_KEY')
    if key is None:
        raise ValueError('SERPER_API_KEY is not set')
    return key


In [ ]:
#| export
@disk_cache.cache
def google_search(q:str,location:str='Austin, Texas',engine:str='google_scholar'):
    """Search the web for information using various search engines.

    This function performs web searches using the SerpAPI client, allowing access to
    information from different search engines including Google Scholar. Results include
    source information for verification.

    Args:
        q (str): The search query string.
        location (str, optional): The location to use for localized search results.
            Defaults to 'Austin, Texas'.
        engine (str, optional): The search engine to use. Defaults to 'google_scholar'.

    Returns:
        dict: A dictionary containing the search results and metadata from the SerpAPI
            response.

    """
    return SerpApiClient({'q':q,'location':location,'engine':engine,'serp_api_key':get_serper_api_key()}).get_dict()

In [ ]:
google_search("how to fix a pipe leak")

{'search_metadata': {'id': '68825f7f02ab8c3fb2c4b4f9',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/93a8d790bf5bbc3f/68825f7f02ab8c3fb2c4b4f9.json',
  'created_at': '2025-07-24 16:29:51 UTC',
  'processed_at': '2025-07-24 16:29:51 UTC',
  'google_scholar_url': 'https://scholar.google.com/scholar?q=how+to+fix+a+pipe+leak&hl=en',
  'raw_html_file': 'https://serpapi.com/searches/93a8d790bf5bbc3f/68825f7f02ab8c3fb2c4b4f9.html',
  'total_time_taken': 0.85},
 'search_parameters': {'engine': 'google_scholar',
  'q': 'how to fix a pipe leak',
  'hl': 'en'},
 'search_information': {'organic_results_state': 'Results for exact spelling',
  'total_results': 40100,
  'time_taken_displayed': 0.11,
  'query_displayed': 'how to fix a pipe leak'},
 'organic_results': [{'position': 1,
   'title': 'Detecting and correcting pipeline leaks before they become a big problem',
   'result_id': 'tUChGR80issJ',
   'link': 'https://www.ingentaconnect.com/content/mts/mtsj/2015/00000049/0

## Simple python code execution


In [ ]:
#| export
from smolagents.local_python_executor import LocalPythonExecutor

In [ ]:
#| export
runner_imports = [
    "collections",
    "datetime",
    "itertools",
    "math",
    "queue",
    "random",
    "re",
    "stat",
    "statistics",
    "time",
    "unicodedata",
    "numpy",
    "pandas",
]


# TODO add state to tools of chat
def run_python_code(code:str):
    """Execute python code, useful when you need to do a calculation or a simple task
    Input should be a valid python code.
    If you want to see the output of a variable, the last line of the code should be that variable name.
    
    Args:
        code (str): Python code to execute
        
    Returns:
        str: Output from executing the code, either the value of the last line or the error message
        
    """
    global runner_imports
    executor = LocalPythonExecutor(additional_authorized_imports=runner_imports)
    try:
        return executor(code).output
    except Exception as e:
        return str(e)

In [ ]:
from stringdale.chat import Chat

In [ ]:
# TODO check why call_tool=True is not working
c = Chat(model='gpt-4o-mini',messages=[{'role':'user','content':'what is the square root of 16?'}],
    tools={'python':run_python_code},
    call_function=True,
    print_prompt=True)
res = await c()
res

[{'role': 'system', 'content': 'choose an appropriate tool to use to answer the following thought based on the following tools:\npython:Execute python code, useful when you need to do a calculation or a simple task\nInput should be a valid python code.\nIf you want to see the output of a variable, the last line of the code should be that variable name.\n\nArgs:\n    code (str): Python code to execute\n\nReturns:\n    str: Output from executing the code, either the value of the last line or the error message'}, {'role': 'user', 'content': 'what is the square root of 16?'}]


{'role': 'assistant',
 'content': {'name': 'python', 'input': {'code': '16 ** 0.5'}, 'output': 4.0},
 'meta': {'input_tokens': 418, 'output_tokens': 30}}

# Export

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()